In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [3]:
from itertools import chain

import nltk
import sklearn
import scipy.stats
#from sklearn.metrics import make_scorer
#from sklearn.cross_validation import cross_val_score
#from sklearn.grid_search import RandomizedSearchCV

import sklearn_crfsuite
#from sklearn_crfsuite import scorers
##from sklearn_crfsuite import metrics


In [4]:
def parse_file(filename):
    f = open(filename, 'r')
    raw = f.readlines()
    sents = []
    s = []
    for line in raw:
        if line.strip():
            tag, token = line.strip().split('\t')
            s.append((token, tag))
        else:
            sents.append(s)
            s = []
    return sents

In [33]:
train_sents = parse_file("pos_train.conll")
test_sents = parse_file("pos_test.conll")

In [36]:
%%time

def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    
    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        #'word[-1:]': word[-1:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        #'postag': postag,
        #'postag[:2]': postag[:2],        
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            #'-1:postag': postag1,
            #'-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True
        
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            #'+1:postag': postag1,
            #'+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True
                
    return features


def get_features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def get_tags(sent):
    return [tag for token, tag in sent]

def get_tokens(sent):
    return [token for token, tag in sent]


X_train = [get_features(s) for s in train_sents]
y_train = [get_tags(s) for s in train_sents]

X_test = [get_features(s) for s in test_sents]
y_test = [get_tags(s) for s in test_sents]


crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs', 
    c1=0.1,
    c2=0.1,
    max_iterations=100, 
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

y_pred = crf.predict(X_test)
same = 0
sum = 0
for st, sp in zip(y_test, y_pred):
    for tt, tp in zip(st, sp):
        if tt == tp:
            same += 1
        sum += 1
print("perc: ", same/sum)

perc:  0.8256021409455843
CPU times: user 11.8 s, sys: 4 ms, total: 11.8 s
Wall time: 11.8 s
